In [ ]:
# Data Extraction from land.ng website 

In [3]:

#Import necessary libaries 

import requests
from bs4 import BeautifulSoup
import pandas as pd
import os


In [ ]:
load the wesite

In [4]:
import requests
from bs4 import BeautifulSoup

URL = "https://land.ng/search-results/?keyword=&states%5B%5D=lagos"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/126.0.0.0 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "en-US,en;q=0.9",
    "Connection": "keep-alive",
    "Upgrade-Insecure-Requests": "1"
}

try:
    response = requests.get(URL, headers=headers, timeout=15)
    print("Status Code:", response.status_code)
except Exception as e:
    print("Error:", e)


Status Code: 500


In [6]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(response.text, "html.parser")
soup


<!DOCTYPE html>

<!--[if lt IE 7]> <html class="no-js ie6 oldie" lang="en-US"> <![endif]-->
<!--[if IE 7]>    <html class="no-js ie7 oldie" lang="en-US"> <![endif]-->
<!--[if IE 8]>    <html class="no-js ie8 oldie" lang="en-US"> <![endif]-->
<!--[if gt IE 8]><!--> <html class="no-js" lang="en-US"> <!--<![endif]-->
<head>
<title>land.ng | 500: Internal server error</title>
<meta charset="utf-8"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
<meta content="noindex, nofollow" name="robots"/>
<meta content="width=device-width,initial-scale=1" name="viewport"/>
<link href="/cdn-cgi/styles/main.css" id="cf_styles-css" rel="stylesheet"/>
</head>
<body>
<div id="cf-wrapper">
<div class="p-0" id="cf-error-details">
<header class="mx-auto pt-10 lg:pt-6 lg:px-8 w-240 lg:w-full mb-8">
<h1 class="inline-block sm:block sm:mb-2 font-light text-60 lg:text-4xl text-black-dark leading-tight mr-2">
<span class="inline-block">I

In [15]:
# -----------------------------------------
# 1. IMPORT LIBRARIES
# -----------------------------------------
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import time
import os

# -----------------------------------------
# 2. HEADERS + BASE URL
# -----------------------------------------
BASE_URL = "https://land.ng/search-results/?keyword=&states%5B%5D=lagos&page="

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/126.0.0.0 Safari/537.36"
}

# -----------------------------------------
# 3. HELPER FUNCTIONS
# -----------------------------------------

def extract_price(price_text):
    """Convert ₦ values to numbers."""
    if not price_text:
        return None

    clean = re.sub(r"[^\d]", "", price_text)
    return int(clean) if clean.isdigit() else None


def extract_land_size(text):
    """Extract land size number from text (sqm, plot, acres)."""
    if not text:
        return None

    match = re.search(r"(\d+(\.\d+)?)", text)
    if match:
        return float(match.group(1))
    return None


def extract_coordinates(text):
    """Find 2 coordinates inside a string."""
    if not text:
        return None, None

    coords = re.findall(r"[-+]?\d*\.\d+", text)
    if len(coords) >= 2:
        return coords[0], coords[1]
    return None, None


# -----------------------------------------
# 4. SCRAPING LOOP (SCRAPES UNTIL NO MORE PAGES)
# -----------------------------------------
all_listings = []
page = 1

while True:
    print(f"🔎 Scraping page {page} ...")

    url = BASE_URL + str(page)
    response = requests.get(url, headers=headers)

    if response.status_code != 200:
        print("❌ Page failed to load. Stopping.")
        break

    soup = BeautifulSoup(response.text, "html.parser")
    listings = soup.find_all("div", class_="item-wrap")

    if not listings:
        print("⛔ No more listings. Finished scraping.")
        break

    for item in listings:
        # Title + Link
        title_tag = item.find("h2", class_="item-title")
        title = title_tag.get_text(strip=True) if title_tag else None
        link = title_tag.find("a")["href"] if title_tag else None

        # Price
        price_tag = item.find("div", class_="item-price")
        price_text = price_tag.get_text(strip=True) if price_tag else None
        price_value = extract_price(price_text)

        # Location
        loc_tag = item.find("address", class_="item-address")
        location = loc_tag.get_text(strip=True) if loc_tag else None

        # Coordinates (if hidden in HTML)
        lat, lon = extract_coordinates(str(item))

        # Land attributes
        land_info = item.find("ul", class_="item-amenities")
        land_size, land_type = None, None

        if land_info:
            info_text = land_info.get_text(" ", strip=True)
            land_size = extract_land_size(info_text)
            land_type = info_text

        # Compute price per sqm
        avg_sqm = None
        if price_value and land_size:
            avg_sqm = round(price_value / land_size, 2)

        # Save record
        all_listings.append({
            "Title": title,
            "Link": link,
            "Price (₦)": price_value,
            "Location": location,
            "Land Size": land_size,
            "Land Type": land_type,
            "Latitude": lat,
            "Longitude": lon,
            "Average ₦/sqm": avg_sqm
        })

    page += 1
    time.sleep(2)

# -----------------------------------------
# 5. SAVE DATASET
# -----------------------------------------
df = pd.DataFrame(all_listings)

save_path = r"C:\Users\THIS-PC\Documents\LagosPropIQ\Data\raw\landsng_lagos_properties.csv"
os.makedirs(os.path.dirname(save_path), exist_ok=True)

df.to_csv(save_path, index=False, encoding="utf-8-sig")

print(f"\n✅ DONE — Total listings collected: {len(df)}")
print(f"📁 Saved to: {save_path}")


🔎 Scraping page 1 ...
🔎 Scraping page 2 ...
🔎 Scraping page 3 ...
🔎 Scraping page 4 ...
🔎 Scraping page 5 ...
🔎 Scraping page 6 ...
🔎 Scraping page 7 ...
🔎 Scraping page 8 ...
🔎 Scraping page 9 ...
🔎 Scraping page 10 ...
🔎 Scraping page 11 ...
🔎 Scraping page 12 ...
🔎 Scraping page 13 ...
🔎 Scraping page 14 ...
🔎 Scraping page 15 ...
🔎 Scraping page 16 ...
🔎 Scraping page 17 ...
🔎 Scraping page 18 ...
🔎 Scraping page 19 ...
🔎 Scraping page 20 ...
🔎 Scraping page 21 ...
🔎 Scraping page 22 ...
🔎 Scraping page 23 ...
🔎 Scraping page 24 ...
🔎 Scraping page 25 ...
🔎 Scraping page 26 ...
🔎 Scraping page 27 ...
🔎 Scraping page 28 ...
🔎 Scraping page 29 ...
🔎 Scraping page 30 ...
🔎 Scraping page 31 ...
🔎 Scraping page 32 ...
🔎 Scraping page 33 ...
🔎 Scraping page 34 ...
🔎 Scraping page 35 ...
🔎 Scraping page 36 ...
🔎 Scraping page 37 ...
🔎 Scraping page 38 ...
🔎 Scraping page 39 ...
🔎 Scraping page 40 ...
🔎 Scraping page 41 ...
🔎 Scraping page 42 ...
🔎 Scraping page 43 ...
🔎 Scraping page 44 .

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))